In [2]:
import torch

In [54]:
t = torch.randn(30,30)
print(t[1].sum())
print(t[1:2,:].sum())
print(t[1].shape)
print(t[1:2,:].shape)

tensor(0.9167)
tensor(0.9167)
torch.Size([30])
torch.Size([1, 30])


In [102]:
import torch

# Inputs
x = torch.randn(12,12,12)
b = torch.randn(12,12)
p = torch.rand((12,12))
B,T,C = x.shape

# Check what indices to keep
keep_ix = b > 0.5 # (B,T), bool
counts = keep_ix.sum(dim=1) # (B,) => How many items to keep per row, max will be padding
Tds = int(counts.max().item()) # Padding index

# Bringing keep_ix==True to the front to discard all other ix
perm = torch.argsort(keep_ix.to(torch.int8), dim=1, descending=True, stable=True)  # (B, T), with index we want to keep in front
sel = perm[:, :Tds]                       # (B, Tds), padded

# Dechunk mask for position & gather chunks
mask_ds = torch.arange(Tds, device=x_hat.device).unsqueeze(0) < counts.unsqueeze(1) # (B, Tds), masking range above count
x_chunks = torch.take_along_dim(x_hat, sel.unsqueeze(-1).expand(-1, -1, E), dim=1) # (B, Tds, E)
P_chunks = torch.take_along_dim(p, sel, dim=1) # (B, Tds)

# Zero-Out padding
x_chunks = x_chunks.masked_fill(~mask_ds.unsqueeze(-1), 0)
P_chunks = P_chunks.masked_fill(~mask_ds, 0)
gather_idx = sel.masked_fill(~mask_ds, 0)     

# State for dechunking
state = {
    "p_full": p, # (B, T)
    "b_full": b, # (B, T)
    "gather_idx": gather_idx, # (B, Tds)
    "mask_ds": mask_ds, # (B, Tds)
}


In [ ]:
B,T = p.shape
att_mask = att_mask if att_mask else torch.ones((B,T))
L = att_mask.sum(dim=1).clamp_min(1).float() # (B,)
F = ((b > 0.5).float() * att_mask.float()).sum(dim=1) / L # ""
G = (p * att_mask.float()).sum(dim=1) / L   # ""
ratio = coef * (((N - 1.0) * F * G) + ((1.0 - F) * (1.0 - G))) # ""
print(ratio.mean())

tensor(1.0304)


In [1]:
import torch
from nhet import DynChunking, HNetConfig

In [2]:
d = DynChunking(512,0.5,HNetConfig)
x = torch.randn((512,512,512))
p,bt=d.route(x)
print(f"bt: {bt.shape}, p: {p.shape}")
x_chunks, P_chunks, mask_ds, state = d.downsample(x,p,bt)
print(f"xchunks: {x_chunks.shape}, pchunks: {P_chunks.shape}, mask_ds: {mask_ds.shape}")

bt: torch.Size([512, 512]), p: torch.Size([512, 512])
xchunks: torch.Size([512, 294, 512]), pchunks: torch.Size([512, 294]), mask_ds: torch.Size([512, 294])
